<a href="https://colab.research.google.com/github/rudysemola/IFTS-MELA-ml-lab/blob/main/timeseries_classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Timeseries classification: Classificazione delle serie temporali usando CNN e RNN

**Cosa Impareremo:** 
- Risolvere un tipico problema di classificazione delle serie temporali per casi d'uso industriali.
- Caricare, visualizzare e standardizzare dati strutturati (serie temporali)
- Costruire, addestrare e valutare modelli di CNN e RNN
- Analizzare la curva di apprendimento

## Introduzione

Questo Lab è un'introduzione alla previsione di serie temporali utilizzando Keras-TensorFlow.

Siamo interessati ad addestrare da zero un classificatore di serie temporali sul set di dati FordA  per un caso d'uso industriale.

In [ ]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

## Load del Dataset FordA

Il set di dati che stiamo usando qui si chiama FordA ([dettagli](http://www.j-wichard.de/publications/FordPaper.pdf)).
Contiene 3601 istanze di training e altre 1320 istanze di test. 
Ogni serie temporale corrisponde a una misurazione del rumore del motore catturata da un sensore. 

Problema: classificazione binaria.
Lo scenario considerato è di un industria automobilistica che vuole effettuare dei test con obiettivo rilevare automaticamente la presenza di un problema specifico con il motore.

Useremo `FordA_TRAIN` file per training e 
`FordA_TEST` file per testing. 

In [ ]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

Vediamo che cosa abbiamo sotto mano 

In [ ]:
print(x_train.shape, y_train.shape, np.min(y_train), np.max(y_train))
print(x_test.shape, y_test.shape, np.min(x_test), np.max(y_test))

Ai fini del laboratorio, riduciamo la dimenzione sia del tr che del test

In [ ]:
x_train, y_train = x_train[0:1200], y_train[0:1200]
x_test, y_test = x_test[0:500], y_test[0:500]

In [ ]:
print(x_train.shape, y_train.shape, np.min(y_train), np.max(y_train))
print(x_test.shape, y_test.shape, np.min(x_test), np.max(y_test))

## Visualizziamo i dati

Qui visualizziamo un esempio di serie temporali per ogni classe nel set di dati

Il modo migliore per visualizzare questi dati è l'uso di grafici (plot).

In [ ]:
classes = np.unique(np.concatenate((y_train, y_test), axis=0))

plt.figure()
for c in classes:
    c_x_train = x_train[y_train == c]
    plt.plot(c_x_train[0], label="class " + str(c))
plt.legend(loc="best")
plt.show()
plt.close()

## Standardizzare (normalizzare) i dati

Le nostre serie temporali sono già in un'unica lunghezza (500). 
Tuttavia, i loro valori sono di solito in vari intervalli. Questo non è l'ideale per una rete neurale; in generale dovremmo cercare di normalizzare i valori di input.

Per questo specifico set di dati, i dati sono già z-normalizzati: ogni campione di serie temporali ha una media uguale a zero e una deviazione standard uguale a uno. *Questo tipo di normalizzazione è molto comune per i problemi di classificazione delle serie temporali.*

Si noti che i dati delle serie temporali utilizzati qui sono univariati, il che significa che abbiamo un solo canale per esempio di serie temporali. Trasformeremo quindi la serie temporale in una multivariata ad un canale utilizzando un semplice rimodellamento tramite numpy. Questo ci permetterà di costruire un modello facilmente applicabile al tempo multivariato serie.

In [ ]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

Per usare `sparse_categorical_crossentropy`, dovremo contare il numero delle classi in anticipo.

In [ ]:
num_classes = len(np.unique(y_train)) # 
num_classes

2

Ora mescoliamo il dataset perché useremo l'opzione `validation_split`durante l'allenamento.

In [ ]:
idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

Normalizza le etichette su numeri interi positivi. Le etichette previste saranno quindi 0 e 1.


In [ ]:
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

## Costruiamo il modello CNN

Adesso costruiremo una Convolutional Neural Network (CNN).

Qui un blocco convoluzionale si basa su tre componenti principali:
1. Un filtro convoluzionale unidimensionale;
2. [Opzionale] Un layer di `batch normalization`
3. Un layer di pooling

I seguenti iperparametri (kernel_size, filters, utilizzo di BatchNorm) sono stati trovati tramite random search utilizzando [KerasTuner](https://github.com/keras-team/keras-tuner).

Vediamo che cosa fanno.


In [ ]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=x_train.shape[1:])

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

**Esercizio** 

Sai trovare un altro modo per costruire il modello?

## Addestriamo il modello

In [ ]:
epochs = 50
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]

In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

In [ ]:
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.25,
    verbose=1,
)

## Valutiamo il modello sui dati di test

In [ ]:
model = keras.models.load_model("best_model.h5")

test_loss, test_acc = model.evaluate(x_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

## Plot della curva di apprendimento 

Analizziamo la learning curve ottenuta

In [ ]:
metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()

**Cosa possiamo osservare?**

Possiamo vedere come l'accuratezza nel train raggiunga (?) dopo (?) epoche. 
Tuttavia, osservando l'accuratezza nel validation, possiamo vedere come la rete (?) 

Oltre la (?)esima epoca, se continuiamo con l'allenamento, l'accuratezza nel  validation  comincerà a diminuire mentre l'accuratezza nel train continuerà ad aumentare: il modello inizia a "overfittare".

(?) Studio ed osservazioni degli studenti


# Esercizi (sugeriti)

- Sperimentare con lo stesso setup presentato la model selection
- Cercare di migliorare le performance (tradeoff con il tempo concesso)
- Provare a verificare il fenomeno dell'overfitting


# Your Turn!

Provare con intero dataset FordA questo lab!
Quanto tempo impiega? Che risultati sei riuscito ad ottenere?